In [ ]:
import numpy as np
import cv2, math
from matplotlib import pyplot as plt

## Image processing

Thresholding done using Otsu's method which is more robust and accurate then normal binary thresholding.

The extraction of the contour is done by Erosion.



In [ ]:
img = cv2.imread("Images\saw_07.png", 0)

ret,img_thresholded = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

kernel = np.ones((3,3),np.uint8)
img_contours = img_thresholded - cv2.erode(img_thresholded, kernel)     # Contours are white, background is black
img_contours_negative = 255 - img_contours                              # Contours are black, background is white

img_cc = img_contours_negative[:,:]
raw_contour = [[i,j] for i in range(img_cc.shape[0]) for j in range(img_cc.shape[1]) if img_cc[i,j] == 0]

## Sorting raw contour points

The *raw_contour* is a list of points that are not ordered in any way both belonging to the base line of the tool and belonging to the teeth profile, so the first step is to separate them. The separation is done by studying the points which have a x coordinate equal to 0, among these points the ones that have the highest y coordinate are the ones belonging to the base line of the tool, the others are the ones belonging to the teeth profile. Once the first points belonging to the base line have been found, it's easy to find all the other points belonging to the base line of the tool by looking for the points that have a y coordinate in a neighborhood of the first points. The idea behind this is that the base line is similar to a straight line. Thus, two lists of points are created, one for the base line and one for the teeth profile. 

The second step is to order the points belonging to the teeth profile in a way that makes sense. The order of the points is important for the Ramer-Douglas-Peucker algorithm to work properly. The points are therefore ordered according to the euclidean distance which allows to find the nearest neighbors of a point which are the next points in the contour. In order to speed up the process, the euclidean distance is not computed for all the points in the teeth profile list but just for the ones that are close enough, it has been empirically proven that a neighborhood dimension of 120 pixels is perfect, to the point which is being studied. 

*sorted_contour* is therefore a list of lists of ordered points structured in this way: [ [teeth_profile], [base_line] ]

In [ ]:
def sort_contour_points(points, neighborhood_dimension, delta_bl=5):

    points = sorted(points, key=lambda p: p[1])
    # 25 sufficiciently large neighborhood to find the first point of the base line
    for i in points[:25]:
        y_max = 0
        if i[1] == 0 and i[0] > y_max:
            y_max = i[0]
            bl_first_point = i

    base_line = [pnt for pnt in points if bl_first_point[0] - delta_bl < pnt[0] < delta_bl + bl_first_point[0]]
    teeth_profile = [pnt for pnt in points if pnt not in base_line]

    # safety check, to handle cases which teeth_profile is has more than one point with x = 0
    x_0 = sorted([pnt for pnt in teeth_profile if pnt[1] == 0], key=lambda p: p[0], reverse=True)
    if len(x_0) > 1:
        teeth_profile = [pnt for pnt in teeth_profile if pnt[1] != 0]
        teeth_profile = x_0 + teeth_profile
    
    splitted_points = [teeth_profile, base_line]
    sorted_contour = []

    for i in splitted_points:
        sorted_points = i
        partial_sorted_contour = [sorted_points[0]]
        sorted_points.remove(sorted_points[0])    
        
        work_point = partial_sorted_contour[0]

        while sorted_points:
            if len(sorted_points) > neighborhood_dimension:
                euclidean_distances = [((work_point[0] - p[0]) ** 2 + (work_point[1] - p[1]) ** 2) for p in sorted_points[:neighborhood_dimension]]
            else:
                euclidean_distances = [((work_point[0] - p[0]) ** 2 + (work_point[1] - p[1]) ** 2) for p in sorted_points]    

            min_distance = min(euclidean_distances)
            nearest_neighbors = [i for i in range(len(euclidean_distances)) if euclidean_distances[i] == min_distance]

            work_point = sorted_points[nearest_neighbors[0]]

            for i in nearest_neighbors:
                partial_sorted_contour.append(sorted_points[i])

            for i in sorted(nearest_neighbors, reverse=True):
                sorted_points.remove(sorted_points[i])
        
        sorted_contour.append(partial_sorted_contour)

            
    return sorted_contour

#sorted_contour(,): list of lists of points, [[teeth_profile], [base_line]]
sorted_contour = sort_contour_points(raw_contour, neighborhood_dimension=120, delta_bl=30)

## Ramer-Douglas-Peucker Algorithm

Since the Ramer-Douglas-Peucker algorithm has been fundamental for the realization of this project, I have decided to write my own function. The function is called *RDP_Algorithm* and it is a recursive function that takes as input a list of points and a tolerance. It returns a list of points that are the result of the RDP algorithm. The function works by finding the point that is the farthest from the line that connects the first and the last point of the list, if the distance is greater than the tolerance, the function is called again with the first half of the list and the second half of the list, otherwise the first and the last point are returned. 

The function *DPTL* takes as input a point and a line and returns the distance between the point and the line. The function works by finding the projection of the point on the line and then calculating the distance between the point and its projection. 

In [ ]:
def RDP_Algorithm(points, epsilon):
    start = points[0]
    end = points[-1]

    dist_point_to_line = DPTL(points, start, end)
    max_value = max(dist_point_to_line)
    max_idx = dist_point_to_line.index(max_value) + 1 #since the first (and the last point) are not included in the calculation

    result = []
    if max_value > epsilon:
        if len(points[:max_idx+1]) == 2:
            result += [list(i) for i in points[:max_idx+1] if list(i) not in result]
        else:
            partial_results_left = RDP_Algorithm(points[:max_idx+1], epsilon)
            result += [list(i) for i in partial_results_left if list(i) not in result]
        if len(points[max_idx:]) == 2:
            result += [list(i) for i in points[max_idx:] if list(i) not in result]
        else:
            partial_results_right = RDP_Algorithm(points[max_idx:], epsilon)
            result += [list(i) for i in partial_results_right if list(i) not in result]
    else:
        result += [points[0], points[-1]]
    
    return result

def DPTL(points, start, end):
    # return a list of distances: distance of each point in points to line formed by start and end
    # compute the angular coefficient and the constant of the line formed by start and end
    # y - mx - q = 0
    a = start[1] - end[1]
    b = end[0] - start[0]
    c = - a*start[0] - b*start[1] 

    return [abs(a*points[i][0]+b*points[i][1]+c)/(math.sqrt(a**2+b**2)) for i in range(1,len(points))]

simplified_teeth_profile = RDP_Algorithm(sorted_contour[0], epsilon = 3)
simplified_base_line = RDP_Algorithm(sorted_contour[1], epsilon = 3)
simplified_contour = [simplified_teeth_profile, simplified_base_line]

## Segmentation of contours into line segments and circular arcs

The functions defined below have been used to segment the teeth profile into circular arcs and straight lines. 

The function *segment_contours* takes as input a threshold, the number of minimum close points in order to detect a circular arc and the list of points belonging to the simplified teeth profile, created by the RDP algorithm. It returns a list of lists of points, each list of points is a segment of the teeth profile. The function works by finding the points that are close enough, distance point to point below a certain threshold, to each other. The points, that are close enough, are then grouped together in a list, close_points. If the number of points in close_points is greater than the minimum number of points, , and the parabola that fits them has an "a" coefficient lesser then 0, it means that the parabola opens downwards which is a common characteristic of all the circular arcs in the tool's blade, then the points are considered to be a circular arc and the group is added to the list of segments. If this is not the case, the points do not belong to a circular arc, so just the point which is being studied is added to the list of segments.

The function *fit_parabola* is designed to fit a parabola to a set of points using the method of least squares. The function takes as input a list of points and returns the "a" coefficient of the parabola that best fit them. The function separates the x and y coordinates of the points into separate arrays, x and y. It then constructs a matrix A where each row corresponds to a point and each column corresponds to a term in the equation of a parabola (y = ax^2 + bx + c). Then by calling calls np.linalg.lstsq, which solves the equation A*coeffs = y in a least squares sense, it finds the coefficients a, b, and c that minimize the sum of the squared residuals. 

The function *extract_circular_arcs* takes as input the list of segments and returns a list of lists of points, each list of points is a circular arc. The function works by finding the groups of points in the *segment_contour* list, that by previous considerations, are the circular arcs present in the teeth profile. Each time a group of points is found, all the points belonging to the *sorted_contour* between the first and the last point of the group, extremes included, are appended, as a list, to *circular_arcs* list which will be returned.

The function *extract_straight_lines* takes as input the list of segments and returns a list of lists of points, each list of points is a straight line. The function works by creating lists of points, which are the straight lines in the teeth profile, between the starting point of the profile and the first point of the first circular arc, between the last point of a circular arc and the first point of the next one and between last point of the last circular arc and the last point in the *sorted_contour* list. In other words, the function works by finding the points, in *sorted_contour*, that are not circular arcs and adding them to the *straight_lines* list which will be returned.

In [ ]:
def fit_parabola(points):
    points_array = np.array(points)

    x = points_array[:, 1]
    y = points_array[:, 0]

    A = np.column_stack((x**2, x, np.ones_like(x)))
    coeffs, _, _, _ = np.linalg.lstsq(A, y, rcond=None)

    return coeffs[0]


def distance(p1, p2):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5


def segment_contours(points, threshold, min_points):
    segments = []
    i = 0
    while i < len(points):
        close_points = [points[i]]
        for j in range(i,len(points)-1):
            if distance(points[j], points[j+1]) < threshold:
                close_points.append(points[j+1])
            else:
                break
        if len(close_points) >= min_points and fit_parabola(close_points) < 0:
            segments.append(close_points)
            i += len(close_points)
        else:
            segments.append([points[i]])
            i += 1
    return segments


def extract_circular_arcs(segment_points, sorted_contour):
    circular_arcs = []
    for i in segment_points:
        if len(i) > 1:
            circular_arcs.append(sorted_contour[sorted_contour.index(i[0]):sorted_contour.index(i[-1])+1]) #so that the last point is included
    return circular_arcs


def extract_straight_lines(segment_points, sorted_contour):
    straight_lines = []
    start_index = 0
    for i in segment_points:
        if len(i) > 1:
            end_index = sorted_contour.index(i[0])
            if end_index - start_index > 0:
                straight_lines.append(sorted_contour[start_index:end_index])
            start_index = sorted_contour.index(i[-1])+1 #so that the last point is not included since it is included in the circular arc segment
    end_index = len(sorted_contour)
    if end_index - start_index > 0:
        straight_lines.append(sorted_contour[start_index:end_index])
    return straight_lines


#segment_contours(,): list of lists, points of the simplified contour that are close to each other are grouped together in a single list 
#[[1° segment], ..., [n° segment], ...] where: 1° segment: [1° point, ..., n° point], ..., n° segment: [1° point, ..., n° point]
segmented_teeth_contour = segment_contours(simplified_contour[0], threshold=35, min_points=3)
circular_arcs = extract_circular_arcs(segmented_teeth_contour, sorted_contour[0])
straight_lines = extract_straight_lines(segmented_teeth_contour, sorted_contour[0])

## Splitting the straight lines into the sides of the teeth

In order to evaluate the ideal profile of each tooth, the straight lines have to be split into the sides of the teeth.  

The function *split_sides* takes as input the list of all points belonging to the straight lines of a tooth profile. *split_sides* works by searching the furthest point from the first point in the list. The code is written such that only the point on the left side and with a lower y coordinate are considered, this speed up the process and makes the code more robust. The point found is then used to split the list of points into two lists, one for each side of the tooth.

*split_curves* takes as input the list of straight lines and returns a list containing the sides of the teeth. The function works by simplifying the lists in straight_lines, therefore each tooth is now represented by a reduced number of points. At this point, a for loop is used to check which of the teeth are "M" shaped since these teeth will be splitted again (from "M" to two normal teeth) and therefore it is needed to find the separation point(s). Generally this kind of tooth are simplifyed by at least 5 ponts and the central points are closer to all the other points then a certain threshold which has been found empirically. Then a for loop is used to study each list of simplifyed points, at this point the problem can be divided in four cases:
1. The tooth is "M" shaped and has two separation points, the function *split_sides* is called and the four lists, two for each sub-teeth, of points are added to the *sides* list.
![M shaped tooth with 2 separation points](ProjectImages\MTooth_6pnts.png)
1. The tooth is "M" shaped and has one separation point, the function *split_sides* is called and the two lists, one for each side, of points are added to the *sides* list.
![M shaped tooth with 1 separation point](ProjectImages\MTooth_5pnts.png)
1. The tooth is apparently not "M" shaped, in this case the function *M_troncated* is called which return "true" if the tooth is "M" shaped but has been troncated or "false" if the tooth is not "M" shaped
    1. The tooth is "M" shaped but has been troncated, the function *split_sides* is called, without the useless points of the troncated "M" side.
    ![M shaped tooth with 1 separation point](ProjectImages\MTooth_Troncated.png)
    1. The tooth is definitely not "M" shaped, in this case the function *split_sides* is called and the two lists of points are added to the *sides* list. 

In [ ]:
def split_sides(points):
    # Find the index of the point with the maximum distance from the first point
    # Check only points on the left and higher (lower y) with respect to the first point
    dists = [np.hypot(p[0]-points[0][0], p[1]-points[0][1]) for p in points if p[0] < points[0][0]]
    max_index = np.argmax(dists)

    points1 = points[:max_index+1]
    points2 = points[max_index:]

    return points1, points2


def M_troncated(points, tooth_threshold=20):
    if abs(points[1][0] - points[2][0]) > tooth_threshold or abs(points[1][1] - points[2][1]) > tooth_threshold:
        return True
    return False


def split_M_tooth(points, sorted_contour, m_threshold, eps):
    split_pnts = []
    for i in range(len(points)):
        if len(points[i]) == 5:
            h = sorted_contour[sorted_contour.index(points[i][1]):sorted_contour.index(points[i][3])+1]
            
            new_simplified_points = RDP_Algorithm(h,eps)
            new_simplified_points.insert(0, points[i][0])
            new_simplified_points.append(points[i][-1])
            points[i] = new_simplified_points

        j = points[i]
        if len(j) > 5:
            poss_split_pnt = []
            for k in j[2:-2]:
                for p in j:
                    if distance(k, p) < m_threshold:
                        dist = True
                    else:
                        dist = False
                        break
                if dist:
                    dist = False
                    poss_split_pnt.append(k)
            split_pnts.append(poss_split_pnt)
    return split_pnts


def split_curves(points, sorted_contour, m_threshold=70):
    RDP_epsilon = 4 
    simplified_straight_lines = []
    m_split_points = []
    splitted_curves = []

    for i in points:
        simplified_straight_lines.append(RDP_Algorithm(i, epsilon=RDP_epsilon))

    m_split_points = split_M_tooth(simplified_straight_lines, sorted_contour, m_threshold, RDP_epsilon-1)
    
    for j in simplified_straight_lines:
        tooth = []
        if len(j) >= 5:
            this_m_split_points = [p for pnt in m_split_points for p in pnt if p in j]
            print(this_m_split_points)
            split_point_index1 = sorted_contour.index(this_m_split_points[0])
            split_point_index2 = sorted_contour.index(this_m_split_points[1])

            tooth.append(sorted_contour[sorted_contour.index(j[0]):sorted_contour.index(j[1])])
            tooth.append(sorted_contour[sorted_contour.index(j[1]):split_point_index1])
                
            splitted_curves.append(tooth)
            tooth = []
            
            tooth.append(sorted_contour[split_point_index2:sorted_contour.index(j[-2])])
            tooth.append(sorted_contour[sorted_contour.index(j[-2]):sorted_contour.index(j[-1])])
        if 2 < len(j) < 5:
            if M_troncated(j, tooth_threshold=30) and len(j) == 4:
                side1, side2 = split_sides(sorted_contour[sorted_contour.index(j[1]):sorted_contour.index(j[-1])])
                tooth.append(side1)
                tooth.append(side2)
            else:
                side1, side2 = split_sides(sorted_contour[sorted_contour.index(j[0]):sorted_contour.index(j[-1])])
                tooth.append(side1)
                tooth.append(side2)
        
        if tooth:
            splitted_curves.append(tooth)
            
    return splitted_curves


#splitted_curves(): list of lists, [[1° tooth], ..., [1° half of the 1° M tooth], [2° half of the 1° M tooth], ...]
#where [one tooth] is [[1° side of the tooth], [2° side of the tooth]]
splitted_curves = split_curves(extract_straight_lines(segmented_teeth_contour, sorted_contour[0]), sorted_contour[0], m_threshold=90)

## Evaluation of the ideal profile of each tooth

In order to obtain the ideal profile of each tooth, the function *fit_triangle* has been defined. The function takes as input two lists of points, the two sides of a tooth, and returns the ideal profile of the tooth. The function works by computing the lines that best fit the two sides of the tooth and then by finding the intersection of the two lines, the vertex. It returns the two lines and their intersection. In the following image the left side of the tooth is shown in green, the right in red and the ideal profile in blue.
![Ideal profile of a tooth](ProjectImages\TriangleFit.png)

To evaluate the quality of the teeth in the tool's blade it's required to check each tooth agle which can be done once the ideal profile of each tooth has been found. To achieve this goal the function *tooth_angle* has been defined. It takes as input the ideal profile of a tooth and returns its angle. The function works by computing the angle between the two lines that best fit the two sides of the tooth. A tooth is considered to be a fine if the angle is greater than 37 degrees and lesser than 42.5 degrees, otherwise it is considered to be defective. The function returns a list of lists whose structure is: [ [ [1° side of the 1° tooth], [2° side of the 1° tooth], [vertex of the 1° tooth], angle_of_the_1°_tooth ], [ ... ], ... ]

In [ ]:
def fit_triangle(points1, points2):
    line1 = np.polyfit([p[1] for p in points1], [p[0] for p in points1], 1)
    line2 = np.polyfit([p[1] for p in points2], [p[0] for p in points2], 1)

    x_intersect = (line2[1] - line1[1]) / (line1[0] - line2[0])
    y_intersect = line1[0] * x_intersect + line1[1]
    vertex = [int(y_intersect), int(x_intersect)] #(y, x) format 

    return vertex, line1, line2


def tooth_angle(points):
    analysis = []
    for i in points:
        v, l1, l2 = fit_triangle(i[0], i[1])
        angle = math.degrees(math.atan(abs((l2[0] - l1[0]) / (1 + l1[0] * l2[0]))))
        analysis.append([i[0],i[1],v,angle])
    return analysis


#blade_inspection: list of lists, [ [[1° side of the 1° tooth], [2° side of the 1° tooth], [vertex of the 1° tooth], angle_of_the_1°_tooth], [...], ... ]
blade_inspection = tooth_angle(splitted_curves)

blank = img.copy()
for i in blade_inspection:
    if 37 < i[3] < 42.5:
        cv2.line(blank, (i[0][0][1], i[0][0][0]), (i[2][1], i[2][0]), 255, 1, cv2.LINE_4)
        cv2.line(blank, (i[1][-1][1], i[1][-1][0]), (i[2][1], i[2][0]), 255, 1, cv2.LINE_4)
        print("Angle:", i[3])
    else:
        start_point = i[0][0]
        middle_point1 = i[0][int(len(i[0])/2)]
        middle_point2 = i[1][int(len(i[1])/2)]
        end_point = i[1][-1]
        print("Start point:", start_point, "End point:", end_point)
        
        #X over the tooth
        #cv2.line(blank, (start_point[1], start_point[0]), (middle_point2[1], middle_point2[0]), 255, 1, cv2.LINE_4)
        #cv2.line(blank, (middle_point1[1], middle_point1[0]), (end_point[1], end_point[0]), 255, 1, cv2.LINE_4)
        
        #Base Line
        #cv2.line(blank, (start_point[1], start_point[0]), (end_point[1], end_point[0]), 255, 1, cv2.LINE_4)

        #Rectangle over the tooth
        #cv2.line(blank, (start_point[1], start_point[0]), (end_point[1], end_point[0]), 255, 1, cv2.LINE_4)
        #cv2.line(blank, (start_point[1], start_point[0]), (start_point[1], i[2][0]), 0, 1, cv2.LINE_4)
        #cv2.line(blank, (start_point[1], i[2][0]), (end_point[1], i[2][0]), 0, 1, cv2.LINE_4)
        #cv2.line(blank, (end_point[1], i[2][0]), (end_point[1], end_point[0]), 0, 1, cv2.LINE_4)
        
        #Circle over the tooth
        #cv2.circle(blank, (i[0][-1][1], i[0][-1][0]), 60, 0, 1)

        print("Tooth number:", blade_inspection.index(i),"Defective Angle:", i[3])

cv2.imwrite("Images\Inspection_Blade_Tool_circle.png", blank)

## Tooth deviation from the ideal profile

To evaluate the deviation of each side from the ideal profile, the function *evaluate_deviation* has been defined. The function takes as input the list of points belonging to the side of a tooth and the ideal profile, which is a line that starts at the vertex of the triangle that fit the tooth and ends either at the first or the last (depending on which side is being analyzed) point of the side, of the tooth. It returns the deviation of the side from the ideal profile. The function works by finding the point in the side that is the farthest from the ideal profile using the *DPTL* function. If this distance is greater than the maximum deviation, 5 pixels, then the tooth is defective otherwise the tooth is not significantly corrupted by burrs or other imperfections.

In [ ]:
def tooth_deviation(analyzed_tooth):
    dev = max(DPTL(analyzed_tooth[0], analyzed_tooth[0][0], analyzed_tooth[2]))
    dev = max(dev, max(DPTL(analyzed_tooth[1], analyzed_tooth[1][-1], analyzed_tooth[2])))
    return dev

for i in blade_inspection:
    deviation = round(tooth_deviation(i))
    if deviation > 5:
        print("Tooth number:", blade_inspection.index(i), "Defective Tooth")
    else:
        print("Tooth number:", blade_inspection.index(i), "\tMax deviation:", deviation)